说明 ：  
0、前一部分卷积处理叫做Feature Extraction，后一部分全连接处理叫做classification

1、每一个卷积核它的通道数量要求和输入通道是一样的。这种卷积核的总数有多少个和你输出通道的数量是一样的。

2、卷积(convolution)后，C(Channels)变，W(width)和H(Height)可变可不变，取决于是否padding。subsampling(或pooling)后，C不变，W和H变。subsampling(或pooling)只在同意Channel内做处理

3、卷积层：保留图像的空间信息。

4、卷积层要求输入输出是四维张量(B,C,W,H)，全连接层的输入与输出都是二维张量(B,Input_feature)。

       PyTorch的nn.Linear（）详解：https://blog.csdn.net/qq_42079689/article/details/102873766

5、卷积(线性变换)，激活函数(非线性变换)，池化；这个过程若干次后，view打平，进入全连接层~

---
代码说明：

1、torch.nn.Conv2d(1,10,kernel_size=3,stride=2,bias=False)

     参数解释： 1是指输入的Channel，灰色图像是1维的；10是指输出的Channel，也可以说第一个卷积层需要10个相同的卷积核；kernel_size=3,卷积核大小是3x3；stride=2进行卷积运算时的步长，默认为1；bias=False卷积运算是否需要偏置bias，默认为False。padding = 0，卷积操作是否补0。

2、self.fc = torch.nn.Linear(320, 10)，这个320获取的方式，可以通过x = x.view(batch_size, -1) # print(x.shape)可得到(64,320),64指的是batch，320就是指要进行全连接操作时，输入的特征维度。

In [1]:
import  torch
from torch.utils.data import DataLoader #我们要加载数据集的
from torchvision import transforms #数据的原始处理
from torchvision import datasets #pytorch十分贴心的为我们直接准备了这个数据集
import  torch.nn.functional as F#激活函数
import torch.optim as optim

batch_size = 64
 #我们拿到的图片是pillow,我们要把他转换成模型里能训练的tensor也就是张量的格式
transform = transforms.Compose([transforms.ToTensor()])

#加载训练集，pytorch十分贴心的为我们直接准备了这个数据集，注意，即使你没有下载这个数据集
#在函数中输入download=True，他在运行到这里的时候发现你给的路径没有，就自动下载
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
#同样的方式加载一下测试集
test_dataset = datasets.MNIST(root='../dataset/mnist/',  train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, shuffle=False,  batch_size=batch_size)

#接下来我们看一下模型是怎么做的
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
         #定义了我们第一个要用到的卷积层，因为图片输入通道为1，第一个参数就是1
         #输出的通道为10，kernel_size是卷积核的大小，这里定义的是5x5的
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        #看懂了上面的定义，下面这个你肯定也能看懂
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        #再定义一个池化层
        self.pooling = torch.nn.MaxPool2d(2)
        #最后是我们做分类用的线性层
        self.fc = torch.nn.Linear(320, 10)
       
    #下面就是计算的过程
    def forward(self, x):
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0) #这里面的0是x大小第1个参数，自动获取batch大小
        #输入x经过一个卷积层，之后经历一个池化层，最后用relu做激活
        x = F.relu(self.pooling(self.conv1(x)))
        #再经历上面的过程
        x = F.relu(self.pooling(self.conv2(x)))
        #为了给我们最后一个全连接的线性层用
        #我们要把一个二维的图片（实际上这里已经是处理过的）20x4x4张量变成一维的
        x = x.view(batch_size, -1) # flatten
        #经过线性层，确定他是0~9每一个数的概率
        x = self.fc(x)
        return x

model = Net()#实例化模型
#把计算迁移到GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

#定义一个损失函数，来计算我们模型输出的值和标准值的差距
criterion = torch.nn.CrossEntropyLoss()
#定义一个优化器，训练模型咋训练的，就靠这个，他会反向的更改相应层的权重
optimizer = optim.SGD(model.parameters(),lr=0.1,momentum=0.5)#lr为学习率


def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):#每次取一个样本
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        #优化器清零
        optimizer.zero_grad()
        # 正向计算一下
        outputs = model(inputs)
        #计算损失
        loss = criterion(outputs, target)
        #反向求梯度
        loss.backward()
        #更新权重
        optimizer.step()
        #把损失加起来
        running_loss += loss.item()
        #每300次输出一下数据
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 2000))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():#不用算梯度
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            outputs = model(inputs)
            #我们取概率最大的那个数作为输出
            _, predicted = torch.max(outputs.data, dim=1)
            total += target.size(0)
            #计算正确率
            correct += (predicted == target).sum().item()
    print('Accuracy on test set: %d %% [%d/%d]' % (100 * correct / total, correct, total))


if __name__=='__main__':
    for epoch in range(10):
        train(epoch)
        if epoch % 10 == 9:
            test()





Using downloaded and verified file: ../dataset/mnist/MNIST\raw\train-images-idx3-ubyte.gz
Extracting ../dataset/mnist/MNIST\raw\train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST\raw

Using downloaded and verified file: ../dataset/mnist/MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ../dataset/mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST\raw

Using downloaded and verified file: ../dataset/mnist/MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ../dataset/mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST\raw



C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,   300] loss: 0.053
[1,   600] loss: 0.017
[1,   900] loss: 0.014
[2,   300] loss: 0.010
[2,   600] loss: 0.009
[2,   900] loss: 0.009
[3,   300] loss: 0.007
[3,   600] loss: 0.007
[3,   900] loss: 0.008
[4,   300] loss: 0.006
[4,   600] loss: 0.006
[4,   900] loss: 0.006
[5,   300] loss: 0.005
[5,   600] loss: 0.005
[5,   900] loss: 0.005
[6,   300] loss: 0.004
[6,   600] loss: 0.004
[6,   900] loss: 0.005
[7,   300] loss: 0.004
[7,   600] loss: 0.004
[7,   900] loss: 0.004
[8,   300] loss: 0.004
[8,   600] loss: 0.003
[8,   900] loss: 0.004
[9,   300] loss: 0.003
[9,   600] loss: 0.004
[9,   900] loss: 0.003
[10,   300] loss: 0.003
[10,   600] loss: 0.003
[10,   900] loss: 0.003
Accuracy on test set: 98 % [9897/10000]
